# Proyecto de pruebas A/B

### Ejercicio
Has recibido una tarea analítica de una tienda en línea internacional. Tus predecesores no consiguieron completarla: lanzaron una prueba A/B y luego abandonaron (para iniciar una granja de sandías en Brasil). Solo dejaron las especificaciones técnicas y los resultados de las pruebas.

- Descripción técnica
    - Nombre de la prueba: recommender_system_test
    - Grupos: А (control), B (nuevo embudo de pago)
    - Fecha de lanzamiento: 2020-12-07
    - Fecha en la que dejaron de aceptar nuevos usuarios: 2020-12-21
    - Fecha de finalización: 2021-01-01
    - Audiencia: 15% de los nuevos usuarios de la región de la UE
    - Propósito de la prueba: probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado
    - Resultado esperado: dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del        producto `(el evento product_page)`, instancias de agregar artículos al carrito de compras `(product_cart)` y compras `(purchase)`. En cada etapa del embudo product_page → product_cart → purchase, habrá al menos un 10% de aumento.
    - Número previsto de participantes de la prueba: 6 000

In [343]:
# Carga de librerias
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

In [344]:
# Carga de datos
marketing_events = pd.read_csv('ab_project_marketing_events_us.csv', sep=',')
events = pd.read_csv('final_ab_events_upd_us.csv', sep=',')
new_users = pd.read_csv('final_ab_new_users_upd_us.csv', sep=',')
participants = pd.read_csv('final_ab_participants_upd_us.csv', sep=',')

### Muestra de datos


### Primer archivo `marketing_events_us.csv`


In [345]:

marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [346]:
#Muestra de datos
marketing_events.head(11)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


Convertimos el tipo de datos de las columnas de `start_dt` y `finish_dt` a datetime


In [347]:
#Convertimos el tipo de datos de las columnas de start_dt y finish_dt a datetime
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

# Comprobamos el tipo de datos de las columnas  
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 580.0+ bytes


In [348]:
#Comprobamos si hay duplicados en el dataframe
marketing_events.duplicated().sum()

np.int64(0)

### Segundo archivo: `events.csv`

In [349]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [350]:
#Muestra de datos
events.sample(11)

,user_id,event_dt,event_name,details
165740,62AF6F5E8CD71BEA,2020-12-16 06:55:12,product_page,NaN
374039,1DAA4A4466D53646,2020-12-22 07:46:15,login,NaN
142460,F8FDA0C2D2124C12,2020-12-12 08:04:48,product_page,NaN
413074,542E93EBC2DA0F34,2020-12-27 07:09:29,login,NaN
376360,000F1B87E2F87740,2020-12-22 02:40:16,login,NaN
328132,2D82D1C3384BD5DC,2020-12-18 16:32:26,login,NaN
124415,485C195C67C2717C,2020-12-08 17:21:28,product_page,NaN
381811,ABC99D63A3451EF5,2020-12-22 20:32:11,login,NaN
143522,5F9155C14FF5C5FA,2020-12-12 01:41:11,product_page,NaN
224470,1DEEE0A91AB6C173,2020-12-24 04:14:59,product_page,NaN


Convertimos el tipo de datos de la columna `event_dt` a datetime


In [351]:
#Convertimos el tipo de datos de las columnas de start_dt y finish_dt a datetime
events['event_dt'] = pd.to_datetime(events['event_dt'])

# Comprobamos el tipo de datos de las columnas  
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [352]:
#analizamos valores unicos de la columna event_name
events['event_name'].unique()

array(['purchase', 'product_cart', 'product_page', 'login'], dtype=object)

Para los valores ausentes en la columna `details` es normal, debido a que no todos los eventos terminaron comprando. Por lo tanto, no es necesario eliminar estos valores ausentes. 

### Tercer archivo: `new_users.csv`

In [353]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [354]:
#Muestra de datos
new_users.sample(11)

,user_id,first_date,region,device
7622,E51E39F1E4BF4567,2020-12-14,APAC,Android
45702,0AFF58884FF696F9,2020-12-12,N.America,iPhone
53968,A1445CD2207E312E,2020-12-13,APAC,PC
2110,320CE96A266DF0CC,2020-12-07,EU,Android
44723,50221A8D91AB25CC,2020-12-12,EU,iPhone
9931,6B3132292D7B6E2D,2020-12-14,EU,Android
56540,6DD4631EDB9E9BB2,2020-12-20,EU,iPhone
11553,B5836DC912FAAB7C,2020-12-21,EU,iPhone
35000,81971E00E433A4AD,2020-12-17,N.America,Mac
36888,85BCB2964422392A,2020-12-17,CIS,iPhone


In [355]:
# validamos datos unicos en la columna de region
new_users['region'].unique()

array(['EU', 'N.America', 'APAC', 'CIS'], dtype=object)

In [356]:
# Creamos una nueva columna para que los datos de region sean mas semanticos
def region(x):
    if x == 'EU':
        return 'UNION EUROPEA'
    elif x == 'N.America':
        return 'NORTE AMERICA'
    elif x == 'APAC':
        return 'ASIA-PACIFICO'
    else:
        return 'COMUNUNIDAD DE ESTADOS INDEPENDIENTES'
new_users['region_s'] = new_users['region'].apply(region)

new_users.sample(11)
    

,user_id,first_date,region,device,region_s
19322,8CAD73CBC0CA4A48,2020-12-08,EU,PC,UNION EUROPEA
36849,87DB490C0D56F3CC,2020-12-17,N.America,Android,NORTE AMERICA
8032,D50CD4AF27FBFAE6,2020-12-14,N.America,iPhone,NORTE AMERICA
33058,CFAC426394D36FA8,2020-12-10,EU,Android,UNION EUROPEA
17142,10E628CE1AE6562D,2020-12-08,APAC,PC,ASIA-PACIFICO
37303,E473C937CD368C92,2020-12-17,EU,PC,UNION EUROPEA
24652,B27BB2F00951DCB8,2020-12-22,EU,iPhone,UNION EUROPEA
55392,FF397AB41B7A8D9C,2020-12-20,N.America,iPhone,NORTE AMERICA
14348,31AFB4FCC5A5CB3C,2020-12-21,EU,PC,UNION EUROPEA
673,9E533297C26C03D1,2020-12-07,EU,PC,UNION EUROPEA


In [357]:
#Revisamos si hay duplicados en el dataframe
new_users.duplicated().sum()

np.int64(0)

### Cuarto archivo: `participants.csv`

In [358]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [359]:
# Muestra de datos
participants.sample(11)

,user_id,group,ab_test
8082,76B98E4196DBD07B,A,interface_eu_test
88,C86F0150DA3B10F2,A,recommender_system_test
2531,0DB417842D3B79A2,A,recommender_system_test
10197,3D182147F0C3EC21,B,interface_eu_test
14501,8155910F11B56B21,A,interface_eu_test
3105,E62E2664B4743026,A,recommender_system_test
13580,ADB92D69EEEA3E9A,B,interface_eu_test
11214,43A01BDD0E721B87,B,interface_eu_test
12634,F2D9CEE188EDF1B3,A,interface_eu_test
13995,71C2BB62F75CABCA,A,interface_eu_test


In [360]:
#Verificamos distribucion entre grupos A y B
group_distribution = participants['group'].value_counts()
print(group_distribution)

group
A    8214
B    6311
Name: count, dtype: int64


In [361]:
#Muestreo de datos para balancear grupos A y B
# Seleccionamos el grupo A y lo balanceamos con el grupo B
group_a = participants[participants['group'] == 'A'].sample(n=len(participants[participants['group'] == 'B']), random_state=42)
balanced_data = pd.concat([group_a, participants[participants['group'] == 'B']])

balanced_data['group'].value_counts()



group
A    6311
B    6311
Name: count, dtype: int64

### Embudo de conversión
- `product_page` → `product_cart` → `purchase`

In [362]:
# Fucionar dataframes
user_data =pd.merge(balanced_data, new_users, on='user_id', how='left')
data = pd.merge(user_data, events, on='user_id', how='left')
data

,user_id,group,ab_test,first_date,region,device,region_s,event_dt,event_name,details
0,D41F3E4CB153371E,A,interface_eu_test,2020-12-19,EU,Android,UNION EUROPEA,2020-12-19 07:34:51,product_page,NaN
1,D41F3E4CB153371E,A,interface_eu_test,2020-12-19,EU,Android,UNION EUROPEA,2020-12-21 01:50:30,product_page,NaN
2,D41F3E4CB153371E,A,interface_eu_test,2020-12-19,EU,Android,UNION EUROPEA,2020-12-23 00:46:12,product_page,NaN
3,D41F3E4CB153371E,A,interface_eu_test,2020-12-19,EU,Android,UNION EUROPEA,2020-12-19 07:34:50,login,NaN
4,D41F3E4CB153371E,A,interface_eu_test,2020-12-19,EU,Android,UNION EUROPEA,2020-12-21 01:50:27,login,NaN
...,...,...,...,...,...,...,...,...,...,...
89053,1D302F8688B91781,B,interface_eu_test,2020-12-15,EU,PC,UNION EUROPEA,2020-12-27 11:12:47,login,NaN
89054,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,UNION EUROPEA,2020-12-14 19:12:49,login,NaN
89055,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,UNION EUROPEA,2020-12-15 01:50:22,login,NaN
89056,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,UNION EUROPEA,2020-12-16 19:27:36,login,NaN


In [363]:
#Grandes numeros
events = len(data)
days = len(data['first_date'].unique())
users = len(data['user_id'].unique())
events_per_user = events / users
events_per_day = events / days

print('KPI SUMMARY')
print('============')
print(f'{events:,} events')
print(f'{days} days')
print(f'{users:,} users')
print(f'{events_per_user:.2f} events per user')
print(f'{events_per_day:.2f} events per day')

KPI SUMMARY
89,058 events
17 days
11,980 users
7.43 events per user
5238.71 events per day


In [364]:
#Tasas de conversión
funnel =(
    data
    .pivot_table(index= 'event_name', values='user_id', aggfunc='nunique')
    .sort_values('user_id', ascending=False)

)

funnel['conv'] = 100*funnel['user_id'] / funnel['user_id'].max()

funnel

,user_id,conv
event_name,,
login,11978,100.000000
product_page,7815,65.244615
purchase,3977,33.202538
product_cart,3882,32.409417


In [365]:
# Calcular conversiones por grupo

def calculate_funnel(group):
    group_users = user_data[user_data['group'] == group]
    total_users = len(group_users)
    
    # Eventos únicos por usuario
    product_page_users = data[(data['group'] == group) & 
                                  (data['event_name'] == 'product_page')]['user_id'].nunique()
    cart_users = data[(data['group'] == group) & 
                           (data['event_name'] == 'product_cart')]['user_id'].nunique()
    purchase_users = data[(data['group'] == group) & 
                              (data['event_name'] == 'purchase')]['user_id'].nunique()
    
    return {
        'product_page': product_page_users / total_users,
        'product_cart': cart_users / product_page_users if product_page_users > 0 else 0,
        'purchase': purchase_users / cart_users if cart_users > 0 else 0,
        'final_conversion': purchase_users / total_users
    }

# Calcular para ambos grupos
funnel_A = calculate_funnel('A')
funnel_B = calculate_funnel('B')

In [366]:
#Generamos tabla de conversiones
table_data = [
    ["1. product_page", 
     f"{funnel_A['product_page']*100:.1f}%", 
     f"{funnel_B['product_page']*100:.1f}%",
     f"+{(funnel_B['product_page']-funnel_A['product_page'])*100:.1f}%",
     f"+{(funnel_B['product_page']-funnel_A['product_page'])/funnel_A['product_page']*100:.1f}%" if funnel_A['product_page'] > 0 else "N/A",
     "No" if (funnel_B['product_page']-funnel_A['product_page'])/funnel_A['product_page'] < 0.1 else "Sí"],
    
    ["2. product_cart", 
     f"{funnel_A['product_cart']*100:.1f}% (de {funnel_A['product_page']*100:.1f}%)", 
     f"{funnel_B['product_cart']*100:.1f}% (de {funnel_B['product_page']*100:.1f}%)",
     f"+{(funnel_B['product_cart']-funnel_A['product_cart'])*100:.1f}%",
     f"+{(funnel_B['product_cart']-funnel_A['product_cart'])/funnel_A['product_cart']*100:.1f}%" if funnel_A['product_cart'] > 0 else "N/A",
     "No" if (funnel_B['product_cart']-funnel_A['product_cart'])/funnel_A['product_cart'] < 0.1 else "Sí"],
    
    ["3. purchase", 
     f"{funnel_A['purchase']*100:.1f}% (de {funnel_A['product_cart']*100:.1f}%)", 
     f"{funnel_B['purchase']*100:.1f}% (de {funnel_B['product_cart']*100:.1f}%)",
     f"+{(funnel_B['purchase']-funnel_A['purchase'])*100:.1f}%",
     f"+{(funnel_B['purchase']-funnel_A['purchase'])/funnel_A['purchase']*100:.1f}%" if funnel_A['purchase'] > 0 else "N/A",
     "No" if (funnel_B['purchase']-funnel_A['purchase'])/funnel_A['purchase'] < 0.1 else "Sí"],
    
    ["Conversión Final", 
     f"{funnel_A['final_conversion']*100:.1f}%", 
     f"{funnel_B['final_conversion']*100:.1f}%",
     f"+{(funnel_B['final_conversion']-funnel_A['final_conversion'])*100:.1f}%",
     f"+{(funnel_B['final_conversion']-funnel_A['final_conversion'])/funnel_A['final_conversion']*100:.1f}%" if funnel_A['final_conversion'] > 0 else "N/A",
     "No" if (funnel_B['final_conversion']-funnel_A['final_conversion'])/funnel_A['final_conversion'] < 0.1 else "Sí"]
]

In [367]:

# Crear DataFrame
funnel_df = pd.DataFrame(table_data, columns=[
    "Etapa", "Grupo A (Control)", "Grupo B (Tratamiento)", 
    "Diferencia (B - A)", "Mejora (%)", "¿Alcanza 10%?"
])

funnel_df

,Etapa,Grupo A (Control),Grupo B (Tratamiento),Diferencia (B - A),Mejora (%),¿Alcanza 10%?
0,1. product_page,63.9%,63.2%,+-0.7%,+-1.1%,No
1,2. product_cart,48.2% (de 63.9%),51.1% (de 63.2%),+2.9%,+6.1%,No
2,3. purchase,106.3% (de 48.2%),98.6% (de 51.1%),+-7.7%,+-7.3%,No
3,Conversión Final,32.7%,31.8%,+-0.9%,+-2.7%,No


### Realizar prueba estatística

In [368]:
def z_test(group_a_events, group_b_events, total_a, total_b):
    p_a = group_a_events / total_a
    p_b = group_b_events / total_b
    p_pool = (group_a_events + group_b_events) / (total_a + total_b)
    z = (p_b - p_a) / np.sqrt(p_pool * (1 - p_pool) * (1/total_a + 1/total_b))
    p_value = stats.norm.sf(abs(z)) * 2  # Two-tailed
    return p_value

# Obtener conteos para pruebas Z
total_a = len(user_data[user_data['group'] == 'A'])
total_b = len(user_data[user_data['group'] == 'B'])

page_a = data[(data['group'] == 'A') & (data['event_name'] == 'product_page')]['user_id'].nunique()
page_b = data[(data['group'] == 'B') & (data['event_name'] == 'product_page')]['user_id'].nunique()

cart_a = data[(data['group'] == 'A') & (data['event_name'] == 'product_cart')]['user_id'].nunique()
cart_b = data[(data['group'] == 'B') & (data['event_name'] == 'product_cart')]['user_id'].nunique()

purchase_a = data[(data['group'] == 'A') & (data['event_name'] == 'purchase')]['user_id'].nunique()
purchase_b = data[(data['group'] == 'B') & (data['event_name'] == 'purchase')]['user_id'].nunique()

# Calcular p-valores
pvals = [
    z_test(page_a, page_b, total_a, total_b),
    z_test(cart_a, cart_b, page_a, page_b),
    z_test(purchase_a, purchase_b, cart_a, cart_b)
]

# Crear tabla de significancia
significance_df = pd.DataFrame({
    'Etapa': ['product_page', 'product_cart', 'purchase'],
    'p-valor': [f"{p:.3f}" for p in pvals],
    '¿Significativo? (α=0.05)': ['✅ Sí' if p < 0.05 else '❌ No' for p in pvals]
})

C:\Users\Te Amo\AppData\Local\Temp\ipykernel_20072\3755627023.py:5: RuntimeWarning: invalid value encountered in sqrt
  z = (p_b - p_a) / np.sqrt(p_pool * (1 - p_pool) * (1/total_a + 1/total_b))


In [369]:

# Estilo para las tablas
def style_table(df):
    return df.style.set_properties(**{
        'text-align': 'center',
        'border': '1px solid black'
    }).set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]
    }])



display(HTML("<h3>Significancia Estadística (Prueba Z)</h3>"))
display(style_table(significance_df))

,Etapa,p-valor,¿Significativo? (α=0.05)
0,product_page,0.405,❌ No
1,product_cart,0.009,✅ Sí
2,purchase,nan,❌ No


### Conclusiones


Conclusiones

- product_cart muestra una diferencia estadísticamente significativa (p = 0.009). Esto indica que hay evidencia suficiente para afirmar que el experimento tuvo un efecto sobre el comportamiento de los usuarios en esta etapa (por ejemplo, agregaron más productos al carrito).

- product_page no presenta significancia estadística (p = 0.405). No se puede concluir que hubo un impacto en esta métrica. Los usuarios probablemente no cambiaron su comportamiento al interactuar con la página de producto.

- purchase tiene un valor nulo (nan), lo que sugiere que no se obtuvo un resultado válido para esta métrica. Esto puede deberse a falta de datos, errores de recolección o segmentación incorrecta.

Recomendaciones 
- Explorar más a fondo el cambio en product_cart: El efecto positivo detectado aquí sugiere que la variación probada tiene potencial para aumentar la intención de compra. Se recomienda analizar qué factor específico (UX, botón, copy, etc.) generó ese cambio.

- Revisar la implementación de la métrica purchase: Es crucial asegurarse de que esta métrica —clave para el negocio— esté correctamente registrada. Repetir la prueba o realizar un control de calidad en la recolección de datos es prioritario.

- No tomar decisiones basadas en product_page por ahora: Como no hay evidencia de impacto en esa etapa, no se justifica un cambio relacionado con la visualización del producto, al menos con la información actual.

- Repetir el experimento con mejoras: Asegúrate de capturar correctamente todas las métricas relevantes. Considera aumentar el tamaño de muestra si la conversión a compra es baja y puede haber limitado la detección de efectos significativos.

